# F-VICE 🏔 ️

In [1]:
import pandas as pd
import os
import plotly.express as px
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# Cargar todos los archivos CSV de la carpeta 'data/' usando polars

dataframes = {}
for filename in os.listdir('data/'):
    if filename.endswith('.csv'):
        filepath = os.path.join('data/', filename)
        df = pd.read_csv(filepath)
        dataframes[filename] = df

# Mostrar los nombres de los archivos cargados
print("Archivos cargados:", list(dataframes.keys()))

Archivos cargados: ['lat_77.96044_lon_-78.5495.csv']


In [3]:
dataframes['lat_77.96044_lon_-78.5495.csv']

,mid_date,v [m/yr],satellite,dt (days)
0,2021-05-19T18:49:19.000Z,974,Sentinel 2,30
1,2019-07-30T07:00:10.000Z,955,Sentinel 2,105
2,2020-02-21T18:39:59.000Z,913,Sentinel 2,310
3,2020-08-14T06:30:19.999Z,648,Sentinel 2,85
4,2020-01-03T06:49:25.000Z,828,Sentinel 2,285
...,...,...,...,...
15077,2023-09-07T18:19:19.000Z,554,Sentinel 2,10
15078,2023-05-03T18:29:19.000Z,1034,Sentinel 2,30
15079,2023-07-06T18:03:05.396Z,1339,Landsat 9,32
15080,2024-05-21T00:39:31.508Z,948,Landsat 8,64


In [8]:
data = dataframes['lat_77.96044_lon_-78.5495.csv']
cols = data.columns.tolist()

data[cols[0]] = pd.to_datetime(data[cols[0]])
data[cols[1]] = data[cols[1]].astype(int)

data_sorted = data.sort_values(by=cols[0])
X = data_sorted[cols[0]]
y = data_sorted[cols[1]]

# Dividir respetando el orden temporal (80% train, 20% test)
split_idx = int(len(X) * 0.7)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]


In [10]:


# Create a DataFrame for plotting
plot_df = pd.DataFrame({
    'date': pd.concat([X_train, X_test]),
    'v [m/yr]': pd.concat([y_train, y_test]),
    'split': ['train'] * len(X_train) + ['test'] * len(X_test)
})

fig = px.scatter(
    plot_df,
    x='date',
    y='v [m/yr]',
    color='split',
    title='Train/Test Split: v [m/yr] over Time',
    opacity=0.7
)
fig.show()

In [ ]:
# Crear y entrenar el modelo XGBoost

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

AttributeError: 'Series' object has no attribute 'reshape'